<a href="https://colab.research.google.com/github/beulah0/GANForMedicalImageReconstruction/blob/main/gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
nih_chest_xrays_data_path = kagglehub.dataset_download('nih-chest-xrays/data')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os

In [ ]:
from PIL import Image
import os

def resize_images(input_folder, output_folder, target_size=(600, 600)):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)


    image_files = [
        f for f in os.listdir(input_folder)
        if f.lower().endswith(('.png', '.jpg', '.jpeg'))
    ]

    for image_file in image_files:
        image_path = os.path.join(input_folder, image_file)
        image = Image.open(image_path).convert("L")

        resized_image = image.resize(target_size)
        output_path = os.path.join(output_folder, image_file)
        resized_image.save(output_path)


input_folder = '/kaggle/input/data/images_001/images'
output_folder = '/kaggle/working/images_re'
target_size = (600, 600)

resize_images(input_folder, output_folder, target_size)


In [ ]:
len(os.listdir('/kaggle/working/images_re'))

In [ ]:
from PIL import Image
import os

def compress_images(source_folder, destination_folder, quality=20):
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    image_files = [f for f in os.listdir(source_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    for image_file in image_files:
        image_path = os.path.join(source_folder, image_file)
        image = Image.open(image_path)


        original_size = os.path.getsize(image_path)


        compressed_image_path = os.path.join(destination_folder, f'{image_file}')
        image.save(compressed_image_path, optimize=True, quality=quality)



source_folder = '/kaggle/working/images_re'
destination_folder = '/kaggle/working/compressed'


compress_images(source_folder, destination_folder, quality=50)



In [ ]:
len(os.listdir('/kaggle/working/compressed'))

In [ ]:
!pip install tqdm --quiet

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
%matplotlib inline

In [ ]:
!ls

In [ ]:
from PIL import Image
import os

def concatenate_images(folder1, folder2, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get a list of all files in the first folder
    image_files_folder1 = [f for f in os.listdir(folder1) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    for image_file1 in image_files_folder1:
        # Open the images from the two folders
        image_path1 = os.path.join(folder1, image_file1)
        image_path2 = os.path.join(folder2, image_file1)

        image1 = Image.open(image_path1)

        # Check if the corresponding image in the second folder exists
        if os.path.exists(image_path2):
            image2 = Image.open(image_path2)

            # Concatenate the images side by side
            concatenated_image = Image.new('RGB', (image1.width + image2.width, max(image1.height, image2.height)))
            concatenated_image.paste(image1, (0, 0))
            concatenated_image.paste(image2, (image1.width, 0))

            # Save the concatenated image to the output folder
            output_path = os.path.join(output_folder, f'concatenated_{image_file1}')
            concatenated_image.save(output_path)


        else:
            print(f"Corresponding image not found for {image_file1}")

# Example usage
source_folder1 = '/kaggle/working/compressed'
source_folder2 = '/kaggle/working/images_re'
destination_folder = '/kaggle/working/joined'

concatenate_images(source_folder1, source_folder2, destination_folder)

In [ ]:
len(os.listdir('/kaggle/working/joined'))

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

def split_images(input_folder, train_folder, test_folder, test_size=0.2, random_state=42):
    if not os.path.exists(train_folder):
        os.makedirs(train_folder)
    if not os.path.exists(test_folder):
        os.makedirs(test_folder)


    image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]


    train_files, test_files = train_test_split(image_files, test_size=test_size, random_state=random_state)


    for file in train_files:
        src_path = os.path.join(input_folder, file)
        dest_path = os.path.join(train_folder, file)
        shutil.copy(src_path, dest_path)


    for file in test_files:
        src_path = os.path.join(input_folder, file)
        dest_path = os.path.join(test_folder, file)
        shutil.copy(src_path, dest_path)

    print(f"Images split into train and test sets. Train folder: {train_folder}, Test folder: {test_folder}")


input_folder = '/kaggle/working/joined'
train_folder = '/kaggle/working/train'
test_folder = '/kaggle/working/test'

split_images(input_folder, train_folder, test_folder, test_size=0.2, random_state=42)

In [ ]:
len(os.listdir('/kaggle/working/test'))

In [ ]:
len(os.listdir('/kaggle/working/train'))

In [ ]:
import glob
train_files = glob.glob('/kaggle/working/train/*.png')
test_files = glob.glob('/kaggle/working/test/*.png')

In [ ]:
len(train_files)

In [ ]:
len(test_files)

In [ ]:
import cv2
for file in train_files[:10]:
    map = cv2.imread(file)
    print (map.shape)

In [ ]:
for k in range(3):
    plt.figure(figsize=(20, 10))
    for j in range(3):
        file = np.random.choice(train_files)
        map = cv2.imread(file)
        map = cv2.cvtColor(map, cv2.COLOR_BGR2RGB)
        plt.subplot(330 + 1 + j)
        plt.imshow(map)
        plt.axis('off')
        #plt.title(trainY[i])
    plt.show()

In [ ]:
maps = []
for j in range(3):
    file = np.random.choice(train_files)
    map = cv2.imread(file)
    map = cv2.cvtColor(map, cv2.COLOR_BGR2RGB)
    maps.append(map)

print ('Input Images')
plt.figure(figsize=(15, 15))
for j, map in enumerate(maps):
    map1 = map[:, :map.shape[1]//2]
    plt.subplot(330 + 1 + j)
    plt.imshow(map1)
    plt.axis('off')
    #plt.title(trainY[i])
plt.show()

print ('Output Images')
plt.figure(figsize=(15, 15))
for j, map in enumerate(maps):
    map2 = map[:, map.shape[1]//2:]
    plt.subplot(330 + 1 + j)
    plt.imshow(map2)
    plt.axis('off')
    #plt.title(trainY[i])
plt.show()

In [ ]:
from tensorflow.keras.layers import LayerNormalization

In [ ]:
import tensorflow

In [ ]:
def encoder_layer(input_layer, filters, bn=True):
    x = tensorflow.keras.layers.Conv2D(filters, kernel_size=(4,4), strides=(2,2), padding='same')(input_layer)
    x = tensorflow.keras.layers.LeakyReLU(alpha=0.2)(x)
    if bn:
        x = LayerNormalization(axis=[1,2])(x)
    return x

def decoder_layer(input_layer, skip_input, filters):
    x = tensorflow.keras.layers.Conv2DTranspose(filters, kernel_size=(4,4), strides=(2,2), padding='same')(input_layer)
    x = tensorflow.keras.layers.Activation('relu')(x)
    x = LayerNormalization(axis=[1,2])(x)
    x = tensorflow.keras.layers.Concatenate()([x, skip_input])
    return x

In [ ]:
source_image_input = tensorflow.keras.layers.Input(shape=(256, 256, 3))

e1 = encoder_layer(source_image_input, 64, bn=False)#64
e2 = encoder_layer(e1, 128)#128
e3 = encoder_layer(e2, 256)#256
e4 = encoder_layer(e3, 512)#512
e5 = encoder_layer(e4, 512)#512
e6 = encoder_layer(e5, 512)#512
e7 = encoder_layer(e6, 512)#512

bottle_neck = tensorflow.keras.layers.Conv2D(512, (4,4), strides=(2,2), padding='same')(e7)
b = tensorflow.keras.layers.Activation('relu')(bottle_neck)

d1 = decoder_layer(b, e7, 512)#512
d2 = decoder_layer(d1, e6, 512)#512
d3 = decoder_layer(d2, e5, 512)#512
d4 = decoder_layer(d3, e4, 512)#512
d5 = decoder_layer(d4, e3, 256)#256
d6 = decoder_layer(d5, e2, 128)#128
d7 = decoder_layer(d6, e1, 64)#64

decoded = tensorflow.keras.layers.Conv2DTranspose(3, kernel_size=(4,4), strides=(2,2), padding='same')(d7)
translated_image = tensorflow.keras.layers.Activation('tanh')(decoded)

generator_network = tensorflow.keras.models.Model(inputs=source_image_input, outputs=translated_image)
print (generator_network.summary())

In [ ]:
def my_conv_layer(input_layer, filters, bn=True):
    x = tensorflow.keras.layers.Conv2D(filters, kernel_size=(4,4), strides=(2,2), padding='same')(input_layer)
    x = tensorflow.keras.layers.LeakyReLU(alpha=0.2)(x)
    if bn:
        x = LayerNormalization(axis=[1,2])(x)
    return x

In [ ]:
source_image_input = tensorflow.keras.layers.Input(shape=(256, 256, 3))
target_image_input = tensorflow.keras.layers.Input(shape=(256, 256, 3))

combined = tensorflow.keras.layers.Concatenate(axis=-1)([source_image_input, target_image_input])

x = my_conv_layer(combined, 64, bn=False)#64
x = my_conv_layer(x, 128)#128
x = my_conv_layer(x, 256)#256
x = my_conv_layer(x, 512)#512

patch_features = tensorflow.keras.layers.Conv2D(1, kernel_size=(4,4), strides=(1,1), padding='same')(x)

discriminator_network = tensorflow.keras.models.Model(inputs=[source_image_input, target_image_input], outputs=patch_features)
print (discriminator_network.summary())

In [ ]:
from tensorflow.keras.optimizers import Adam


adam_optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
discriminator_network.compile(loss='mse', optimizer=adam_optimizer, metrics=['accuracy'])

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Lambda


source_image_input = Input(shape=(256, 256, 3), name="source_image_input")


g_output = generator_network(source_image_input)


discriminator_network.trainable = False
d_output = discriminator_network([source_image_input, g_output])


d_output = Lambda(lambda x: x, name="discriminator_output")(d_output)
g_output = Lambda(lambda x: x, name="l1_output")(g_output)


pix2pix = Model(inputs=source_image_input, outputs=[d_output, g_output])

pix2pix.summary()


In [ ]:
pix2pix.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-4, beta_1=0.5),
    loss={
        "discriminator_output": "mse",
        "l1_output": "mae"
    },
    metrics={"discriminator_output": "accuracy"}
)


In [ ]:
def get_predictions(input_sample, generator_network):
    input_sample = np.expand_dims(input_sample, axis=0)
    output_sample = generator_network.predict_on_batch(input_sample)
    return output_sample

def get_generated_samples(generator_network, maps_input):
    generated_samples = generator_network.predict_on_batch(maps_input)
    return generated_samples

def get_map_samples(batch_size):
    random_files = np.random.choice(train_files, size=batch_size)
    maps_input = []
    maps_output = []
    for file in random_files:
        map = cv2.imread(file)
        map = cv2.cvtColor(map, cv2.COLOR_BGR2RGB)
        map1 = map[:, :map.shape[1]//2]
        map2 = map[:, map.shape[1]//2:]
        map1 = cv2.resize(map1, (256, 256))
        map2 = cv2.resize(map2, (256, 256))
        maps_input.append((map1-127.5)/127.5)
        maps_output.append((map2-127.5)/127.5)
    maps_input = np.array(maps_input)
    maps_output = np.array(maps_output)
    return maps_input, maps_output
def show_generator_results(generator_network):
    maps = []
    for j in range(3):
        file = np.random.choice(test_files)
        map = cv2.imread(file)
        map = cv2.cvtColor(map, cv2.COLOR_BGR2RGB)
        maps.append(map)

    print ('Input Images')
    plt.figure(figsize=(13, 13))
    for j, map in enumerate(maps):
        map1 = map[:, :map.shape[1]//2]
        map1 = cv2.resize(map1, (256, 256))
        plt.subplot(330 + 1 + j)
        plt.imshow(map1)
        plt.axis('off')
        #plt.title(trainY[i])
    plt.show()

    print ('Predicted Output Images')
    plt.figure(figsize=(13, 13))
    for j, map in enumerate(maps):
        map1 = map[:, :map.shape[1]//2]
        map1 = cv2.resize(map1, (256, 256))
        map1 = (map1-127.5)/127.5
        output = get_predictions(map1, generator_network)[0]
        output = (output+1.0)/2.0
        plt.subplot(330 + 1 + j)
        plt.imshow(output)
        plt.axis('off')
        #plt.title(trainY[i])
    plt.show()

In [ ]:
import tensorflow as tf

In [ ]:
import tensorflow as tf

epochs = 200
batch_size = 1
steps = 187

for i in range(epochs):
    if i % 5 == 0:
        show_generator_results(generator_network)

    for j in range(steps):
        # Get batch of training samples
        maps_input, maps_output = get_map_samples(batch_size=batch_size)
        maps_input = tf.convert_to_tensor(maps_input, dtype=tf.float32)
        maps_output = tf.convert_to_tensor(maps_output, dtype=tf.float32)

        # Generate fake output
        generated_maps_output = get_generated_samples(generator_network, maps_input)
        generated_maps_output = tf.convert_to_tensor(generated_maps_output, dtype=tf.float32)

        # Create real/fake labels as tensors
        fake_patch = tf.zeros((batch_size, 16, 16, 1), dtype=tf.float32)
        real_patch = tf.ones((batch_size, 16, 16, 1), dtype=tf.float32)

        # --- Train Discriminator ---
        discriminator_network.trainable = True
        loss_d_real = discriminator_network.train_on_batch([maps_input, maps_output], real_patch)
        loss_d_fake = discriminator_network.train_on_batch([maps_input, generated_maps_output], fake_patch)

        # Average loss
        if isinstance(loss_d_real, (list, tuple)):
            loss_d = [(loss_d_real[0] + loss_d_fake[0]) / 2,
                      (loss_d_real[1] + loss_d_fake[1]) / 2]
        else:
            loss_d = (loss_d_real + loss_d_fake) / 2.0

        # --- Train Generator ---
        discriminator_network.trainable = False

        # Make sure your generator model outputs are named correctly
        # If your Pix2Pix model outputs are unnamed, just pass a list of tensors
        loss_g = pix2pix.train_on_batch(maps_input,{
        "discriminator_output": real_patch,
        "l1_output": maps_output
        },
    return_dict=True
    )

        # # Print losses every 100 steps
        # if j % 100 == 0:
        #     if isinstance(loss_d, list):
        #         print(f"Epoch:{i}, Step:{j}, D-Loss:{loss_d[0]:.3f}, D-Acc:{loss_d[1]*100:.3f}, G-Loss:{loss_g:.3f}")
        #     else:
        #         print(f"Epoch:{i}, Step:{j}, D-Loss:{loss_d:.3f}, G-Loss:{loss_g:.3f}")
        if j % 100 == 0:
            print(
            f"Epoch:{i}, Step:{j}, "
            f"D-Loss:{loss_d[0]:.3f}, "
            f"G-Loss:{loss_g['loss']:.3f}, "
            f"GAN:{loss_g['discriminator_output_loss']:.3f}, "
            f"L1:{loss_g['l1_output_loss']:.3f}"
            )


In [ ]:
for i in range(5):
    show_generator_results(generator_network)

In [ ]:
pip install scikit-image

In [ ]:
import numpy as np
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

def image_quality_metrics(real, generated):
    real = np.clip(real, 0, 1)
    generated = np.clip(generated, 0, 1)

    psnr = peak_signal_noise_ratio(real, generated, data_range=1.0)
    ssim = structural_similarity(real, generated, channel_axis=-1, data_range=1.0)

    return psnr, ssim


In [ ]:
real_img = maps_output[0].numpy()
gen_img = generated_maps_output[0].numpy()

psnr, ssim = image_quality_metrics(real_img, gen_img)

print(f"PSNR: {psnr:.2f} dB")
print(f"SSIM: {ssim:.4f}")


In [ ]:
quality_score = (ssim * 100)
quality_score